# Structured Output
结构化输出允许代理以特定的、可预测的格式返回数据。你无需解析自然语言响应，而是直接获得应用程序可以直接使用的 JSON 对象、`Pydantic` 模型或数据类等结构化数据。

LangChain 的 `create_agent` 会自动处理结构化输出。用户设置所需的结构化输出模式，当模型生成结构化数据时，它会被捕获、验证，并以代理状态中的 `'structured_response'` 键返回。


<img src="./assets/LC_StructuredOutput.png" width="700">


## 初始化
加载并检查所需的环境变量

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env, doublecheck_pkgs

# 从 .env 加载环境变量
load_dotenv()

# 检查并打印结果
doublecheck_env(".env")  # 检查环境变量

DASHSCOPE_API_KEY=****931f
DASHSCOPE_BASE_URL=****e/v1
LANGSMITH_API_KEY=****ef8f
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****s-56
LANGSMITH_ENDPOINT=****.com


## 结构化输出示例

In [2]:
from langchain_qwq import ChatQwen
import os
model=ChatQwen(
    model="qwen3-max", 
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url=os.getenv("DASHSCOPE_BASE_URL"),
    )

In [3]:
from typing_extensions import TypedDict

from langchain.agents import create_agent


class ContactInfo(TypedDict):
    name: str
    email: str
    phone: str


agent = create_agent(model=model, response_format=ContactInfo)

recorded_conversation = """We talked with John Doe. He works over at Example. His number is, let's see, 
five, five, five, one two three, four, five, six seven. Did you get that?
And, his email was john at example.com. He wanted to order 50 boxes of Captain Crunch."""

result = agent.invoke(
    {"messages": [{"role": "user", "content": recorded_conversation}]}
)

result["structured_response"]

{'name': 'John Doe', 'email': 'john@example.com', 'phone': '5551234567'}

支持多种数据类型
* pydantic `BaseModel`
* `TypedDict`
* `dataclasses`
* JSON 模式（dict）

In [6]:
from langchain.agents import create_agent
from pydantic import BaseModel, Field


class ContactInfo(BaseModel):
    """用户信息结构"""
    name: str = Field(..., description="姓名")
    email: str = Field(..., description="电子邮件")
    phone: str = Field(..., description="电话号码")


agent = create_agent(model,response_format=ContactInfo)

recorded_conversation = """ We talked with John Doe. He works over at Example. His number is, let's see, 
five, five, five, one two three, four, five, six seven. Did you get that?
And, his email was john at example.com. He wanted to order 50 boxes of Captain Crunch."""

result = agent.invoke(
    {"messages": [{"role": "user", "content": recorded_conversation}]}
)

# 访问结构化响应
result["structured_response"]

ContactInfo(name='John Doe', email='john@example.com', phone='5551234567')

Pydantic 的 `Field(..., description="...") `主要好处是 自动生成高质量的工具描述、API 文档和模型提示 ，让 LLM 更好地理解字段含义，提高工具调用准确率。

最推荐使用 pydantic，原因如下:

|特性	|TypedDict	|Pydantic BaseModel|
|----|----|----|
|类型检查	|运行时无验证	|运行时验证 + IDE 提示|
|验证	|❌ 无	|✅ 内置验证（正则、长度、格式）|
|序列化	|手动	|model_dump() / model_dump_json()|
|文档生成	|❌ 手动	|✅ 自动（Field description）|
|错误处理	|静默失败	|✅ 抛出 ValidationError|